## Description
This notebook is a demo of a speaker anonymization system. 

Installation of the toolkit takes around **20min** to complete on Colab.

Scroll down to the bottom if you simply want to listen to an example.

In [1]:
# Go to Menu > Runtime > Change runtime.
# Change hardware acceleration to GPU.

# This will take ~20min to complete
!git clone https://github.com/deep-privacy/sa-toolkit
!cd sa-toolkit && ./install.sh

Cloning into 'sa-toolkit'...
remote: Enumerating objects: 3972, done.
remote: Counting objects: 100% (3972/3972), done.
remote: Compressing objects: 100% (1858/1858), done.
remote: Total 3972 (delta 2187), reused 3579 (delta 1802), pack-reused 0
Receiving objects: 100% (3972/3972), 86.37 MiB | 15.17 MiB/s, done.
Resolving deltas: 100% (2187/2187), done.
 == Google colab / Kaggle detected, running py37 | Warning: Performing /usr/local OVERWRITE! ==
Using local $CUDAROOT: /usr/local/cuda
Cuda version: 111
 - Downloading a pre-compiled version of kaldi
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0   2859      0 --:--:-- --:--:-- --:--:--  2859
100   133  100   133    0     0    815      0 --:--:-- --:--:-- --:--:--   815
100   651  100   651    0     0   2657      0 --:--:-- --:--:-- --:--:--  2657
100  927M  100  927M    0     0  27.7M      0  0:

In [12]:
%%bash
cd sa-toolkit/pkwrap/egs/librispeech/v1
wget -q https://huggingface.co/Champion/SA-models/resolve/main/e2e_tdnnf_wav2vec_fairseq_hibitrate.zip
yes no | unzip ./e2e_tdnnf_wav2vec_fairseq_hibitrate.zip

wget -q https://huggingface.co/Champion/SA-models/resolve/main/e2e_tdnnf_wav2vec_fairseq_hibitrate_vq_48.zip
yes no | unzip ./e2e_tdnnf_wav2vec_fairseq_hibitrate_vq_48.zip

wget -q https://huggingface.co/Champion/SA-models/resolve/main/e2e_tdnnf.zip
yes no | unzip ./e2e_tdnnf.zip

wget -q https://huggingface.co/Champion/SA-models/resolve/main/decode.zip
yes no | unzip decode.zip

rm *.zip
cd -

cd sa-toolkit/pkwrap/egs/LJSpeech
wget -q https://huggingface.co/Champion/SA-models/resolve/main/hifigan_w2w2.zip
yes no | unzip hifigan_w2w2.zip

wget -q https://huggingface.co/Champion/SA-models/resolve/main/hifigan_wav2vec2_vq_48.zip
yes no | unzip hifigan_wav2vec2_vq_48.zip

wget -q https://huggingface.co/Champion/SA-models/resolve/main/hifigan_tdnnf.zip
yes no | unzip hifigan_tdnnf.zip

rm *.zip
cd -

wget -q https://huggingface.co/Champion/SA-models/resolve/main/LibriTTS_f0_stats.zip
yes no | unzip LibriTTS_f0_stats.zip
rm LibriTTS_f0_stats.zip

Archive:  hifigan_wav2vec2_vq_48.zip
  inflating: exp/hifigan_w2w2_vq_48/g_00045000  
/content


In [3]:
import sys, os
from pathlib import Path
PYTHON_PACKAGE="/usr/local/lib/python3.7/site-packages/" # ATM: Colab is currently running on python3.7
# import python package installed with miniconda
sys.path.insert(0,PYTHON_PACKAGE)
for egg in list(Path(PYTHON_PACKAGE).glob(f"**/*egg-link")):
  sys.path.append(open(egg).read().splitlines()[0])

# TODO: Set LD_LIBRARY_PATH accordingly to  sa-toolkit/env.sh and kaldi path.sh
# os.environ['LD_LIBRARY_PATH'] = open("sa-toolkit/env.sh").read().splitlines()[0].split("LD_LIBRARY_PATH=")[1].split(";")[0]
# os.environ['LD_LIBRARY_PATH'] += f":{os.getcwd()}/sa-toolkit/kaldi/src/lib"
# os.environ['KALDI_ROOT'] = f":{os.getcwd()}/sa-toolkit/kaldi/"

In [4]:
import torch
import torchaudio
import numpy as np
import pkwrap
import pkwrap.infer_helper as demo
import json
from IPython.display import Audio, HTML, Javascript

In [6]:

# forward_asr_wav2vec2, pk_model = demo.init_asr_model(
#     model="local/chain/e2e/tuning/tdnnf_wav2vec_fairseq_hibitrate.py",
#     exp_path="exp/chain/e2e_tdnnf_wav2vec_fairseq_hibitrate/",
# )
# forward_asr_wav2vec2, pk_model = demo.init_asr_model(
#     model="local/chain/e2e/tuning/tdnnf.py",
#     exp_path="exp/chain/e2e_tdnnf/",
# )
forward_asr_wav2vec2, pk_model = demo.init_asr_model(
    model="local/chain/e2e/tuning/tdnnf_wav2vec_fairseq_hibitrate_vq.py",
    exp_path="exp/chain/e2e_tdnnf_wav2vec_fairseq_hibitrate_vq_48/",
    pkwrap_vq_dim=48,
)

Loading 'exp/chain/e2e_tdnnf_wav2vec_fairseq_hibitrate_vq_48/final.pt'


CRITICAL:root:pkwrap: -- Failed to import kaldi you better not be in training mode (no backward possible) --


In [13]:
# forward_synt, synt_model = demo.init_synt_hifigan_w2v2(
#     model=f"local/tuning/hifi_gan_wav2vec2.py",
#     exp_path=f"exp/hifigan_w2w2",
#     asr_bn_model=pk_model,
#     model_weight="g_00050000",
#     json_stats_file="stats.json"
# )
# forward_synt, synt_model = demo.init_synt_hifigan_w2v2(
#     model=f"local/tuning/hifi_gan_tdnnf.py",
#     exp_path=f"exp/hifigan_tdnnf",
#     asr_bn_model=pk_model,
#     model_weight="g_00111000",
#     json_stats_file="stats.json"
# )
forward_synt, synt_model = demo.init_synt_hifigan_w2v2(
    model=f"local/tuning/hifi_gan_wav2vec2.py",
    exp_path=f"exp/hifigan_w2w2_vq_48",
    asr_bn_model=pk_model,
    model_weight="g_00045000",
    json_stats_file="stats.json"
)

Loading 'exp/hifigan_w2w2_vq_48/g_00045000'


In [22]:
from google.colab.output import eval_js
import base64
import tempfile

# Test with your voice
if False:
  display(Javascript(demo.record_audio_colab))
  output = eval_js('recordAudio({})')
  tmpf = tempfile.NamedTemporaryFile(suffix='.wav')
  tmpf.write(base64.b64decode(output))
  print('Recording saved to:', tmpf.name, "\nNow running ffmpeg..")
  tmpfwav = tempfile.NamedTemporaryFile(suffix='.wav').name
  get_ipython().getoutput(f"ffmpeg -i {tmpf.name} -ac 1 -f wav {tmpfwav} -y -hide_banner -loglevel panic")
  tmpf.close()
  waveform, sr = torchaudio.load(tmpfwav)
  waveform = torchaudio.transforms.Resample(orig_freq=sr,
                                                    new_freq=16000)(waveform)
  display(Audio(data=waveform.cpu(), rate=16000))
  text_gt = None

In [ ]:
# Or some file on the disk
if False:
  file = "own.mp3"
  tmpfwav = tempfile.NamedTemporaryFile(suffix='.wav').name
  get_ipython().getoutput(f"ffmpeg -i {file} -ac 1 -f wav {tmpfwav} -y -hide_banner -loglevel panic")
  waveform, sr = torchaudio.load(tmpfwav)
  waveform = torchaudio.transforms.Resample(orig_freq=sr,
                                                    new_freq=16000)(waveform)
  display(Audio(data=waveform.cpu(), rate=16000))
  text_gt = None

In [15]:
# Or use libri speaker
if True:
  !wget -q https://huggingface.co/proxy-datasets-preview/assets/librispeech_asr/--/clean/validation/0/audio/audio.mp3 -O audio.mp3
  waveform, sr = torchaudio.load("audio.mp3")
  text_gt = "HE WAS IN A FEVERED STATE OF MIND OWING TO THE BLIGHT HIS WIFE'S ACTION THREATENED TO CAST UPON HIS ENTIRE FUTURE "

In [16]:
# ASR result from the PPG extractor
display(Audio(data=waveform, rate=16000))
out, net = forward_asr_wav2vec2(waveform.to(demo.device).clone())
text = demo.kaldi_asr_decode(out)
if text_gt != None:
  _,_, repport = demo.computeWER(text, text_gt)
  print(text, "\n")
  display(HTML(repport))

HE WAS IN A FEVERED STATE OF MIND OWING TO THE BLIGHT HIS WIFE'S ACTION THREATENED TO CAST UPON HIS ENTIRE FUTURE 



In [26]:
#@title Select the target for voice conversion
f0_stats = json.loads(open("./stats.json",'r').readline())
spkids = list(f0_stats.keys())
target = '6081' #@param ['7190', '4481', '8324', '7402', '5808', '8123', '1447', '3240', '6081', '8088', '125', '831', '4788', '6385', '8838', '1069', '7226']

# (AWGN) Add additive white gaussian noise to the F0
# disable by default
rand_pitch = False


def _norm(f0, f0_stats, filename):
  # F0 norm
  pitch = pkwrap.hifigan.f0.m_std_norm(f0, f0_stats[target], filename)

  if rand_pitch:
      # Set a target channel noise power to something very noisy
      target_noise_db = 15
      # Convert to linear Watt units
      target_noise_watts = 10 ** (target_noise_db / 10)
      # Generate noise samples
      mean_noise = 0
      noise_volts = np.random.normal(
          mean_noise, np.sqrt(target_noise_watts), len(pitch)
      )
      ii = pitch == 0
      pitch = pitch + torch.tensor(noise_volts, dtype=pitch.dtype)
      pitch[ii] = 0
  return pitch
pkwrap.hifigan.f0.set_norm_func(_norm)

f0 = pkwrap.hifigan.f0.get_f0(
    waveform.cpu().clone(), f0_stats=f0_stats
).to(demo.device)

audio = forward_synt(audio=waveform.to(demo.device).clone(), f0=f0, target=[target])
display(Audio(data=audio, rate=16000))

/usr/local/lib/python3.7/site-packages/amfm_decompy/pYAAPT.py:983: RuntimeWarning: invalid value encountered in true_divide
  phi[lag_min:lag_max] = formula_nume/np.sqrt(formula_denom)
